In [ ]:
CONFIG_PATH = '../../config.json'
SPLIT = 'test'
IOU_THRESHOLD = 0.7

In [2]:
# Load config
import json
import numpy as np
from pathlib import Path

### Подготавливаем визуализацию

In [3]:
# Create temporary directory for predictions
pred_labels_dir = Path('runs/segment/predict/labels')
pred_images_dir = Path('runs/segment/predict')

In [4]:
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

In [5]:
from matplotlib import pyplot as plt

## Плотный тест

In [ ]:
import pickle
with open('../../groups/gt_test_dense.pkl', 'rb') as f:
    gt_groups = pickle.load(f)
with open('../../groups/pred_test_dense.pkl', 'rb') as f:
    pred_groups = pickle.load(f)

In [ ]:
import sys
sys.path.append('../../utils')
sys.path.append('../..')

In [8]:
from interpolate.refine_markup_by_yolo import get_box
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

In [9]:
def make_square_box(box, padding=10):
    x1, y1, x2, y2 = box
    width = x2 - x1
    height = y2 - y1
    
    # Make square by taking larger dimension
    size = max(width, height)
    
    # Calculate center
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    
    # Calculate new box coordinates with padding
    new_x1 = int(center_x - size/2 - padding)
    new_y1 = int(center_y - size/2 - padding)
    new_x2 = int(center_x + size/2 + padding)
    new_y2 = int(center_y + size/2 + padding)
    
    return np.array([new_x1, new_y1, new_x2, new_y2])

In [ ]:
IMAGES_DIR = Path('../../data/dense_test/imgs')
PADDING = 0.2

In [ ]:
output_dir = Path('../../data/dense_test/gt_groups')
output_dir.mkdir(exist_ok=True)

for group_idx, group in tqdm(enumerate(gt_groups), total=len(gt_groups)):
    group_dir = output_dir / str(group_idx)
    group_dir.mkdir(exist_ok=True)
    
    for obj_idx, obj in enumerate(group):
        box = get_box(obj['obj'])
        box = np.round(box).astype(int)
        p = round(max(box[2] - box[0], box[3] - box[1]) * PADDING)
        
        box = make_square_box(box, p)
        
        img_path = obj['img']
        
        img = cv2.imread(str(IMAGES_DIR / img_path.name))
        
        padded = cv2.copyMakeBorder(
            img, 
            p, p, p, p, 
            cv2.BORDER_CONSTANT, 
            value=[0,0,0]
        )
        cropped = padded[
            max(p+box[1], 0):
            max(p+box[3], 0), 
            max(p+box[0], 0):
            max(p+box[2], 0)
        ]
        if min(cropped.shape[:2]) < 5:
            print(cropped.shape, p+box[1], p+box[3], p+box[0], p+box[2])
        output_path = group_dir / f"{obj_idx}.png"
        cv2.imwrite(str(output_path), cropped)

 36%|███▌      | 444/1241 [02:06<01:52,  7.08it/s]

(2, 2, 3) 998 1000 375 377


 36%|███▌      | 446/1241 [02:06<01:50,  7.19it/s]

(4, 5, 3) 998 1003 401 406


 43%|████▎     | 536/1241 [02:19<01:27,  8.09it/s]

...

 80%|███████▉  | 992/1241 [04:18<01:47,  2.31it/s]

(1, 2, 3) -1 1 774 776


 96%|█████████▌| 1187/1241 [05:29<00:14,  3.62it/s]

(1, 1, 3) 999 1000 445 446


100%|██████████| 1241/1241 [05:45<00:00,  3.59it/s]
